In [ ]:
import itchat
import numpy as np
import os
#from collections import Counter
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']#绘图时可以显示中文
plt.rcParams['axes.unicode_minus']=False#绘图时可以显示中文
#from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
import re

In [ ]:
itchat.auto_login(hotReload = True)
friends = itchat.get_friends(update = True)


In [ ]:
import pandas as pd
sex=list(map(lambda x:x['Sex'],friends[1:]))
nickname=list(map(lambda x:x['NickName'],friends[1:]))
signature=list(map(lambda x:x['Signature'],friends[1:]))
province=list(map(lambda x:x['Province'],friends[1:]))
city=list(map(lambda x:x['City'],friends[1:]))
headimg=list(map(lambda x:x['HeadImgUrl'],friends[1:]))
info={"nickname":nickname,"sex":sex,"province":province,"city":city,"signature":signature,'headimgurl':headimg}
data=pd.DataFrame(info)
data.head(10)

In [ ]:
from pyecharts import Bar
from collections import Counter
city_5=Counter(city).most_common(5)
# 返回出现次数最多的20条
bar = Bar('好友所在城市TOP5', '', title_pos='center', width=800, height=500)
attr, value = bar.cast(city_5)
bar.add('', attr, value, is_visualmap=True, visual_text_color='#fff', is_more_utils=True,is_label_show=True)
bar

### 好友性别分析

In [ ]:
from pyecharts import Pie
sexs=list(map(lambda x:x['Sex'],friends[1:])) #提取好友性别
value = [sexs.count(1), sexs.count(2), sexs.count(0)]#对性别进行计数
sex_attr=['male','female','unknown']
pie = Pie('好友性别比例', '好友总人数：%d' % len(sex), title_pos='center')
pie.add('', sex_attr, value, radius=[30, 75], rosetype='area', is_label_show=True,is_legend_show=True, legend_top='bottom')
# pie.show_config()
pie
#pie.render('好友性别比例.html')

### 好友地理位置分析

In [ ]:
from pyecharts import Map
provinces_count = data.groupby('province', as_index=True)['province'].count().sort_values()
attr = list(map(lambda x: x if x != '' else '未知', list(provinces_count.index)))#未填写地址的改为未知
value = list(provinces_count)
map_1 = Map("微信好友位置分布图",title_pos="center",width=1000, height=500)
map_1.add('', attr, value, is_label_show=True, is_visualmap=True, visual_text_color='#000',visual_range=[0,120])
map_1.render(path='/Users/ls/Desktop/微信好友分布.html')

In [ ]:
from pyecharts import Map
def get_attr(friends, key):
    return list(map(lambda user: user.get(key), friends))

def fun_pos(friends):
    users = dict(province=get_attr(friends, "Province"),
                 city=get_attr(friends, "City"),
                 nickname=get_attr(friends, "NickName"))
    provinces = pd.DataFrame(users)
    provinces_count = provinces.groupby('province', as_index=True)['province'].count().sort_values()
    attr = list(map(lambda x: x if x != '' else '未知', list(provinces_count.index)))#未填写地址的改为未知
    value = list(provinces_count)
    map_1 = Map("微信好友位置分布图",title_pos="center",width=1000, height=500)
    map_1.add('', attr, value, is_label_show=True, is_visualmap=True, visual_text_color='#000',visual_range=[0,120])
    map_1.render(path='/Users/ls/Desktop/好友分布.html')
fun_pos(friends)

In [ ]:
import wordcloud
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
import jieba
import jieba.analyse
import snownlp
from pyecharts import Bar
signatures = ''
emotions = []
for friend in friends:
    signature = friend['Signature']
    if signature != None:
        signature = signature.strip().replace("span","").replace("class","").replace("emoji","")#去除无关数据
        signature = re.sub(r'1f(\d.+)',"",signature)
        
    if len(signature) > 0:
        nlp = snownlp.SnowNLP(signature)
        emotions.append(nlp.sentiments)#nlp.sentiments：权值
        signatures += " ".join(jieba.analyse.extract_tags(signature,5)) #关键字提取
back_coloring = np.array(Image.open("1.jpg"))#图片可替换
word_cloud2 = WordCloud(font_path = 'zt.ttf',
                        background_color = 'white',
                        max_words = 1200,
                        mask = back_coloring,
                        margin = 15)
word_cloud2.generate(signatures)
image_colors = ImageColorGenerator(back_coloring)
plt.figure(figsize=(6,5),dpi=160)
plt.imshow(word_cloud2.recolor(color_func=image_colors))
plt.axis("off")
plt.show()
word_cloud2.to_file("signatures.jpg")
#人生观
count_positive = len(list(filter(lambda x:x>0.66,emotions)))#大于0.66为积极
count_negative = len(list(filter(lambda x:x<0.33,emotions)))#小于0.33为消极
count_neutral = len(list(filter(lambda x:x>=0.33 and x <= 0.66,emotions)))
value=[count_positive,count_negative,count_neutral]
att_attr=['积极','消极','中性']
bar=Bar('个性签名情感分析',title_pos='center',width=800,height=500)
bar.add('', att_attr, value, visual_range=[0, 200], is_visualmap=True, is_label_show=True)
    #bar.add(",att_attr,value,visual_range=[0, 200], is_visualmap=True, is_label_show=True)
bar

In [ ]:
#下载好友头像
basePath = os.path.abspath('.')
baseFolder = basePath + '\\HeadImages\\'
if(os.path.exists(baseFolder) == False):
    os.makedirs(baseFolder)

# Analyse Images
image_tags = ''
for index in range(1,2):
    friend = friends[index]
# Save HeadImages
    imgFile = baseFolder + '\\Image%s.jpg' % str(index)
    imgData = itchat.get_head_img(userName = friend['UserName'])
    if(os.path.exists(imgFile) == False):
        with open(imgFile,'wb') as file:
             file.write(imgData)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
img=Image.open("/Users/ls/Desktop/numpydemo/weixin/faces.jpg")
plt.figure("好友头像")
plt.imshow(img)
plt.show()

In [ ]:
from aip import AipFace
import base64
APP_ID  = "17777086"
API_KEY = "VqGadasdibjzadsEFabRgnGZESLCs"  #请替换为自己的apikey，注册地址https://login.bce.baidu.com/?account=
SECRET_KEY = "WcZwZPQymL6Ba9WS4U8zAO0cemj6FUN3"
client = AipFace(APP_ID, API_KEY, SECRET_KEY) #初始化aipface对象

In [ ]:
filepath = "/Users/ls/Desktop/numpydemo/weixin/faces.jpg"
with open(filepath, "rb") as fp:
    base64_data = base64.b64encode(fp.read())
image = str(base64_data, 'utf-8')
imageType = "BASE64"
options = {}
options["face_field"] = "age,gender,beauty,expression"
options["max_face_num"] = 10
options["face_type"] = "LIVE"
result = client.detect(image, imageType, options)
#print(result)

In [ ]:
import pandas as pd
facelist=result['result']['face_list']
output=pd.DataFrame(facelist)
output.drop(['angle','face_token'],axis=1,inplace=True)
output.head()

In [ ]:
import cv2
import matplotlib.pyplot as plt
img=cv2.imread('faces.jpg')
location=result['result']['face_list'][0]['location']
left_top=(location['left'],location['top'])
right_bottom=(left_top[0]+location['width'],left_top[1]+location['height'])
cv2.rectangle(img,(282,219),(328,264),(255,0,0),2)#周迅
cv2.rectangle(img,(206,40),(246,80),(255,0,0),2) #杜鹃
cv2.rectangle(img,(344,40),(384,85),(255,0,0),2)#李宇春
cv2.rectangle(img,(154,190),(194,230),(255,0,0),2)#章子怡
cv2.rectangle(img,(87,119),(125,159),(255,0,0),2)#李冰冰
#cv2.imshow('img',img)
plt.imshow(img)
plt.show()